#Motor de búsqueda semántica
Instalamos las siguientes librerias:
* Pytorch: https://pytorch.org/
* Sentence transformers: https://www.sbert.net/
* Sci-kit learn : https://scikit-learn.org/
* Plotly-express : https://plotly.com/python/plotly-express/

In [1]:
!pip install torch
!pip install sentence-transformers
!pip install scikit-learn
!pip install plotly-express

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Importamos las librerias instaladas y cargamos el modelo de codificación de textos (embeddings).
El modelo a emplear para este caso será:
> https://huggingface.co/intfloat/multilingual-e5-large-instruct

Parámetros del modelo:
* Memoria: 1.2 gbs
* Input length: 512 tokens
* Output size embeddings: 1024


In [2]:
from sentence_transformers import SentenceTransformer, util
import time
import torch
from sklearn.manifold import TSNE
import plotly.express as px
import numpy as np
from google.colab import drive

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
model = SentenceTransformer('intfloat/multilingual-e5-large-instruct',device="cuda")
print(model)
docs = []
doc_emb = None
DEVICE ="cuda" #or "cpu"
drive.mount('/content/drive')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Mounted at /content/drive


Definimos las principales funciones:
- **loadDoc(name)** : Lee un documento de texto y lo fragmenta por párrafos (_"\n"_), posteriormente codifica cada fragmento con el modelo en embeddings. Genera una lista de textos con cada párrafo, y otra lista con cada uno de los párrafos codificado.
- **showEmbeddings(docs_emb, query_emb)** : Representa los embeddings generados en un espacio bidimensional, mediante el algoritmo de reducción no lineal t-sne.
- **semantic_search(query)**: Codifica la consulta con el modelo definido, y realiza una búsqueda, comparando cada uno de los embeddings de texto con el de la consulta, mediante la simulitud del coseno. Devuelve los 5 textos con mayor similitud.

In [13]:
def get_detailed_instruct(query: str) -> str:
    task_description = 'Given a web search query, retrieve relevant passages that answer the query'
    return f'Instruct: {task_description}\nQuery: {query}'

def tSNE_reduction(embeddings):
    print("Input shape",embeddings.shape)
    tsne_model = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200, metric = 'cosine')
    #tsne_model = TSNE(n_components=2, random_state=42,metric = 'cosine')
    tsne_embeddings_values = tsne_model.fit_transform(embeddings)
    print("Output shape",tsne_embeddings_values.shape)
    return tsne_embeddings_values

def loadDoc(name):
    text = ""
    with open(name,'r', encoding='utf8') as input:
        file_content = input.readlines()
        for line in file_content:
            if len(line) > 1 and line[-2] != ".":
                line = line[:-1]
            text = text + line

    docs = text.split("\n")
    docs = [i.replace('\r\n', '') for i in docs if len(i.strip())>0]    #clean
    print("Number of paragraphs: ",len(docs))
    start = time.process_time()
    doc_emb = model.encode(docs, normalize_embeddings=True, show_progress_bar=True, device=DEVICE, batch_size=16)
    end = time.process_time()
    print("Processing time:",end - start)
    return docs, doc_emb

def semantic_search(text):
    #Encode query
    start = time.process_time()
    query_emb = model.encode([get_detailed_instruct(text)], normalize_embeddings=True, show_progress_bar=True, device=DEVICE)
    hits = util.semantic_search(query_emb, doc_emb, top_k=5, score_function=util.cos_sim)

    end = time.process_time()
    print("Processing results time:", end - start)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print("(Score: {:.4f})".format(hit['score']), docs[hit['corpus_id']])
    return query_emb

def showEmbeddings(embeddings_array, query_embeddings):
    concatenated = np.concatenate((embeddings_array,query_embeddings), axis=0)
    embeddings_values = tSNE_reduction(concatenated)
    colors = ["paragraph" for i in docs]
    colors.append("query")

    names = ["paragraph_" + str(i) for i,item in enumerate(docs)]
    names.append("query")
    fig = px.scatter(embeddings_values, x=0, y=1, color=0)

    fig = px.scatter(
        x = embeddings_values[:,0],
        y = embeddings_values[:,1],
        hover_name = names,
        title = 'Paragraphs representation', width = 800, height = 600,
        color = colors
    )
    fig.show(renderer="colab")


Leemos el documento de texto *convenioIter.txt*, que es un documento estructurado, donde la información semántica está contenida en párrafos. Cada parrafo (delimitador "\n") se extrae y se codifica con el modelo. Se muestra el tiempo de procesamiento total.

In [14]:
#%cd /content/drive/My Drive/UOC/Docs/
#!ls "/content/drive/My Drive/UOC/Docs"

docs, doc_emb = loadDoc("/content/drive/My Drive/UOC/Docs/convenioIter.txt")

Number of paragraphs:  81


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Processing time: 3.4983491819999983


Realizamos la búsqueda con la consulta introducida, para lo cual se codifica  con el modelo y se realiza la comparación con los demás embeddings de texto en base a la similitud del coseno. Se devuelve una lista con los párrafos ordenada por mayor similitud (*score*)

In [15]:
query = "¿Cuántos días de vacaciones tengo?"

print(query)
query_emb = semantic_search(query)

¿Cuántos días de vacaciones tengo?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing results time: 0.0634882129999994
(Score: 0.8769)  j) Los trabajadores con al menos un año de antigüedad en la empresa, tendrán derecho a cuatro (4) días de licencia con sueldo, sin la obligación de la posterior justificación, por asuntos propios, y siempre que sea compatible con la organización de la empresa. Tales días deben ser usados dentro del año natural que corresponda.
(Score: 0.8700) Artículo 24.- Vacaciones. Las vacaciones anuales retribuidas tendrán una duración de 21 días hábiles, siempre y cuando este cómputo equivalga al menos a 30 días naturales. El período de vacaciones preferente será el mes de agosto, salvo acuerdo en sentido contrario. Para el período de disfrute, desacuerdo respecto a éste y calendario de disfrute se estará a lo establecido en el artículo 38 del TRLET. Al personal que disfrute de sus vacaciones durante la totalidad de tres semanas naturales en agosto; así como al personal especialmente designado por la empresa para permanecer trabajando tr

Mostramos todos los párrafos en el espacio bidimensional, aplicando el algoritmo t-sne, para apreciar mejor la similitud de cada uno con respecto a la consulta realizada.(Score)


In [10]:
showEmbeddings(doc_emb,query_emb)

Input shape (83, 1024)
Output shape (83, 2)
